In [ ]:
!source $HOME/physics/bin/activate

In [1]:
import sys
sys.path.append('../..')
from utils.rootLoad import rootLoad
rootLoad()

Shared objects loaded.
Now you can use ROOT in this notebook.


In [1]:
from ROOT import TFile, TH1D, TCanvas, TLegend, TLatex, TF1, gStyle, TGraphErrors, TMultiGraph
import uproot
import pandas as pd
import yaml


Welcome to JupyROOT 6.28/06


In [2]:
inTree = uproot.open('../Data/run175174828_230428174901_preprocessed.root')['PreprocessedData']
inPixels = [inTree[f'pixel{i}'].arrays(library='pd') for i in range(16)]
innerPixels = [inTree[f'pixel{i}'].arrays(library='pd') for i in [5, 6, 9, 10]]

In [3]:
outFile = TFile('../Data/output/spectrumVisual.root', 'recreate')
dirAmpl = outFile.mkdir('amplitude')
elecAmpl = outFile.mkdir('electrons')

In [ ]:
def amplitudeSpectrumAndFits(inData, outDir, logFile, ipixel):

    gStyle.SetOptStat(0)

    h = TH1D(f'h_{ipixel}', 'h', 100, 1, 101)
    for x in inData['amplitude']:   h.Fill(x)
    h.SetFillColorAlpha(797, 0.5)
    h.SetLineColor(797)
    h.SetFillStyle(3356)
    h.SetTitle(f'Pixel {ipixel}; Signal amplitude (mV); Counts')

    with open(logFile, 'r') as f:   log = yaml.load(f, Loader=yaml.FullLoader)

    kAfit = TF1(f'kAfit_{ipixel}', '[0] * exp(-(x-[1])*(x-[1])/(2*[2]*[2]))', log[f'pixel{ipixel}']['fits'][0]['range'][0], 
                                                                    log[f'pixel{ipixel}']['fits'][0]['range'][1])
    kAfit.SetParameters(log[f'pixel{ipixel}']['fits'][0]['param'][0], 
                        log[f'pixel{ipixel}']['fits'][0]['param'][1], 
                        log[f'pixel{ipixel}']['fits'][0]['param'][2])
    kAfit.SetLineColor(860)
    kAfit.SetLineWidth(2)

    kBfit = TF1(f'kBfit_{ipixel}', '[0] * exp(-(x-[1])*(x-[1])/(2*[2]*[2]))', log[f'pixel{ipixel}']['fits'][1]['range'][0], 
                                                                    log[f'pixel{ipixel}']['fits'][1]['range'][1])
    kBfit.SetParameters(log[f'pixel{ipixel}']['fits'][1]['param'][0],
                        log[f'pixel{ipixel}']['fits'][1]['param'][1],
                        log[f'pixel{ipixel}']['fits'][1]['param'][2])
    kBfit.SetLineColor(419)
    kBfit.SetLineWidth(2)

    h.Fit(kAfit, 'RM+')
    print(f'Pixel {ipixel} kAfit: {kAfit.GetChisquare()} / {kAfit.GetNDF()}')
    h.Fit(kBfit, 'RM+')
    print(f'Pixel {ipixel} kBfit: {kBfit.GetChisquare()} / {kBfit.GetNDF()}')

    outDir.cd() 
    h.Write()
    kAfit.Write()
    kBfit.Write()

    return h, kAfit, kBfit


In [ ]:
spectra = []
kAfits = []
kBfits = []

for ipixel, pixel in zip([5, 6, 9, 10], innerPixels):
    h, kAfit, kBfit = amplitudeSpectrumAndFits(pixel, dirAmpl, f'../src/config/simpleFitCfg.yml', ipixel)
    spectra.append(h)
    kAfits.append(kAfit)
    kBfits.append(kBfit)

In [ ]:
canvas = TCanvas('canvas', '^{55}Fe spectrum', 800, 600)
canvas.Divide(2, 2)
legends = []

for i, h in enumerate(spectra):
    frame = canvas.cd(i+1).DrawFrame(1, 0, 101, 1200, 'Pixel '+str(innerPixels[i]['pixel'][0])+'; Signal amplitude (mV); Counts')
    h.Draw('same')
    kAfits[i].Draw('same')
    kBfits[i].Draw('same')
    frame.GetYaxis().SetTitleSize(0.05)
    frame.GetYaxis().SetTitleOffset(1.)
    frame.GetXaxis().SetTitleSize(0.05)

    TLatex().SetTextSize(0.04)
    TLatex().SetTextFont(42)
    TLatex().DrawLatexNDC(0.2, 0.75, '^{55}Fe acquisition')
    TLatex().DrawLatexNDC(0.2, 0.7, 'K_{#alpha} = '+f'({kAfits[i].GetParameter(1):.1f} #pm {kAfits[i].GetParameter(2):.1f}) mV')
    TLatex().DrawLatexNDC(0.2, 0.65, 'K_{#beta} = '+f'({kBfits[i].GetParameter(1):.1f} #pm {kBfits[i].GetParameter(2):.1f}) mV')

    canvas.cd(i+1)
    legend = TLegend(0.2, 0.3, 0.5, 0.6)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.06)
    legend.SetTextFont(42)

    legend.AddEntry(h, 'Data', 'f')
    legend.AddEntry(kAfits[i], 'K_{#alpha} fit', 'l')
    legend.AddEntry(kBfits[i], 'K_{#beta} fit', 'l')
    
    legends.append(legend)
    legends[i].Draw()


canvas.SaveAs('../Data/output/spectrumVisual.pdf')

In [4]:
canvas = TCanvas('canvas', 'Calibration curve', 800, 600)

graphs = []
fits = []
legends = []
colors = [860, 419, 797, 633]

inFile = TFile.Open('../Data/output/calibrationOutput.root')

for i, ipixel in enumerate([5, 6, 9, 10]):

    graph = inFile.Get(f'calibrationPx{ipixel}')
    fit = graph.GetFunction(f'calibrationPx{ipixel}Fit')
    #fit = inFile.Get(f'calibrationPx{ipixel}Fit')

    graph.SetMarkerStyle(20)
    #graph.SetMarkerSize(1.)
    graph.SetMarkerColor(colors[i]-1)
    graph.SetTitle(f'Pixel {innerPixels[i]["pixel"][0]}; Energy (keV); Amplitude (mV)')
    graph.SetLineColor(colors[i]-1)

    fit.SetLineColor(colors[i])
    fit.SetLineWidth(2)

    graphs.append(graph)
    fits.append(fit)

canvas.cd()

mg = TMultiGraph()
mg.SetTitle('Energy calibration; Energy (keV); Amplitude (mV)')
for graph in graphs:    mg.Add(graph)
mg.Draw('AP')
for fit in fits:    fit.Draw('same')

legend = TLegend(0.2, 0.5, 0.5, 0.75)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.SetTextSize(0.04)
legend.SetTextFont(42)

for i, graph in enumerate(graphs):    legend.AddEntry(graph, f'Pixel {innerPixels[i]["pixel"][0]}', 'pl')

legend.Draw('same')

canvas.SaveAs('../Data/output/calibrationVisual.pdf')

Info in <TCanvas::Print>: pdf file ../Data/output/calibrationVisual.pdf has been created
